In [2]:
from transformers import Trainer, TrainingArguments, HfArgumentParser

In [3]:
parser = HfArgumentParser(TrainingArguments)

In [6]:
parser.parse_args()

usage: ipykernel_launcher.py [-h] --output_dir OUTPUT_DIR
                             [--overwrite_output_dir [OVERWRITE_OUTPUT_DIR]]
                             [--do_train [DO_TRAIN]] [--do_eval [DO_EVAL]]
                             [--do_predict [DO_PREDICT]]
                             [--evaluation_strategy {no,steps,epoch}]
                             [--prediction_loss_only [PREDICTION_LOSS_ONLY]]
                             [--per_device_train_batch_size PER_DEVICE_TRAIN_BATCH_SIZE]
                             [--per_device_eval_batch_size PER_DEVICE_EVAL_BATCH_SIZE]
                             [--per_gpu_train_batch_size PER_GPU_TRAIN_BATCH_SIZE]
                             [--per_gpu_eval_batch_size PER_GPU_EVAL_BATCH_SIZE]
                             [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                             [--eval_accumulation_steps EVAL_ACCUMULATION_STEPS]
                             [--learning_rate LEARNING_RATE]
               

SystemExit: 2

/home/keller/.conda/envs/huggingface_latest/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
